# Introduction au package KERAS et au "Deep-Learning": réseaux de neurones convolutifs  (CNNs) & reconnaissance d'images

Dans ce notebook nous allons apprendre à manipuler les fonctions de base du [package Keras](https://keras.io/) permettant de mettre en oeuvre des algorithmes de réseaux de neurones, dits de "deep learning". Nous considérerons pour cela le problème le plus classique : catégorisation d'images par réseaux de neurones convolutifs (CNNs : Convolutional Neural Networks).
Nous travaillerons sur la base MNIST qui présente l'avantage (1) d'être de taille relativement restreinte, ce qui permet de réaliser l'analyse dans un temps raisonnable, et (2) d'être déjà disponible dans Keras, ce qui permet de gagner du temps sur des questions de mise en forme des données.

Le plan sera le suivant  :
1. **préparation des données**
2. **réseaux de neurones à une couche & régression (logistique) multinomiale**
3. **réseaux de neurones multicouches & modèles non-linéaires**
4. **réseaux de neurones convolutifs & apprentissage de représentation**

On commence par charger les packages et par bloquer la graine du générateur aléatoire par souci de reproductibilité. 
**Attention : keras faisant partie de tensorflow depuis la version 2 de tensorflow, les imports ne seront pas les mêmes si vous travaillez à partir de la version 2 ou d'une version antérieure.**
Pour cela, renseigner la variable **"tensorflow_version"** dans la cellule ci-dessous à la valeur **"V1"** ou **"V2"** en fonction de votre configuration.

In [ ]:
# specify tensorflow version
tensorflow_version = "V2"

In [ ]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
# import tensorflow requirements - depending on version
if tensorflow_version == "V1":
    from keras import backend as K
    from keras.models import Model
    from keras.models import Sequential
    from keras.models import load_model
    from keras.layers import Dense, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras.utils import to_categorical
if tensorflow_version == "V2":
    from tensorflow.keras.models import Model
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.models import load_model
    from tensorflow.keras.layers import Dense, Activation, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.utils import to_categorical
# set random seed
np.random.seed(123)

# Partie 1. Préparation des données

Nous commençons par charger le jeu de données sauvegardé par le premier notebook.

In [ ]:
f = open('mnist-dataset.pickle', 'rb')
dataset = pickle.load(f)
f.close()

In [ ]:
X_train = dataset[0]
X_test  = dataset[1]
Y_train = dataset[2]
Y_test  = dataset[3]
y_train = dataset[4]
y_test  = dataset[5]

# Partie 2. Réseaux de neurones à une couche & régression logistique multinomiale

Voir notebook précedent.
On se contente de charger le modèle sauvegardé.

In [ ]:
linreg = load_model("model_multinomial")

# Partie 3. Réseaux de neurones multicouches & modèles non-linéaires

Le formalisme des réseaux de neurones permet de construire de véritables réseaux mettant en jeu des successions de couches. La couche d'entrée correspond aux variables d'entrée (elle mettra donc ici en jeu 784 neurones) et les couches suivantes impliquent des neurones tels que celui illustré précédemment.

Dans le cas le plus classique, le réseau est totalement connecté ("fully connected") : chaque neurone est connecté à tous les neurones de sa couche précédente et tous les neurones de sa couche de sortie. On parle parfois de "perceptron multi-couches" (MLP : multi-layer perceptron).

<img src="pic/mlp.png" width="600">

Chaque neurone d'une couche interne définit un score par combinaison linéaire de ses variables d'entrée (variables initiales ou sorties des neurones de la couche précédente), et lui applique une fonction d'activation.
Les fonctions d'activations "historiques" sont des fonctions sigmoidales (telles que la fonction logistique ou la fonction tangente hyperbolique), mais depuis quelques années la fonction "ReLu" (pour Rectified Linear Unit) semble faire l'unanimité dans la communauté : 

<img src="pic/activation.png" width="400">

Cette fonction consiste simplement à conserver le score du neurone inchangé s'il est positif mais le passer à zéro s'il est négatif.

De manière générale, grâce à l'utilisation de ces fonctions d'activation (que ce soit de type sigmoïdale ou ReLU), les réseaux de neurones à plusieurs couches sont capables de construire des fonctions de prédiction non linéaires (par opposition au cas d'un neurone isolé ou d'une couche unique comme précédemment).


Pour implémenter un réseau de neurones à plusieurs couches, il suffit avec Keras de créer une succession de couches de type "Dense".

Avant cela, il nous faut néanmoins **transformer les données originales** (des images) pour en avoir une représentation vectorielle, comme on l'a fait pour la régression multinomiale. On commence donc par transformer chaque image de taille 28x28 en un vecteur de taille 784,

In [ ]:
input_dim = 28*28 
X_train_vec = X_train.reshape(60000, input_dim) 
X_test_vec  = X_test.reshape(10000, input_dim)

### <span style="color:red"> $\Rightarrow$ EXERCICE: définir le réseau à deux couches cachées mettant en jeu 64 et 100 neurones pour réaliser le problème multi-classee initial, en vous inspirant de l'exemple donné en cours.</span> 

In [ ]:
# TODO : define multi-layer perceptron with two hidden layers of 64 and 100 neurons to adress the multiclass problem
mlp = Sequential()


On note que comme précédemment on doit spécifier les dimensions des données d'entrée pour la première couche du réseau. En revanche, les dimensions des couches internes sont determinées automatiquement.

On peut alors afficher le "résumé" du réseau via la méthode *summary* : 

In [ ]:
# TODO : show summary of the model and check output shapes & number of parameters involved in the various layers


On vérifie par exemple que le nombre de paramètres de la 2ème couche est bien conforme à ce qui est attendu : elle met en jeu 100 neurones connectés aux 64 neurones de la première couche, soit 100 x (64+1) = 6500 paramètres (le +1 correspond aux termes de biais).

Il suffit alors d'appliquer la même procédure que précédemment : compiler le modèle et l'apprendre.

In [ ]:
mlp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
history_mlp = mlp.fit(X_train_vec, Y_train, batch_size=32, epochs=10, verbose=1, validation_split=0.2)

On peut alors comparer les performances des deux modèles : 

In [ ]:
score_linreg = linreg.evaluate(X_test_vec, Y_test, verbose=0)
score_mlp = mlp.evaluate(X_test_vec, Y_test, verbose=0)
print("multinomial regression test-set accuracy =", score_linreg[1])
print("multi-layer perceptron test-set accuracy =", score_mlp[1])

Là aussi, on peut constater que l'apprentissage n'a peut-être pas été suffisamment long.

In [ ]:
plt.figure(figsize=(8,4))
# show accuracy
plt.subplot(1,2,1)
plt.plot(history_mlp.history['accuracy'], label = 'train')
plt.plot(history_mlp.history['val_accuracy'], label = 'validation')
plt.title("accuracy vs epochs")
plt.ylim(0.85,0.98)
plt.legend()
# show loss
plt.subplot(1,2,2)
plt.plot(history_mlp.history['loss'], label = 'train')
plt.plot(history_mlp.history['val_loss'], label = 'validation')
plt.title("loss vs epochs")
plt.ylim(0.1,0.5)
plt.legend()
plt.show()

## <span style="color:red"> $\Rightarrow$ EXERCICE: Reproduire cette analyse en considérant un nombre plus important d'epochs. Le risque de sur-apprentissage est-il plus fort qu'avec la régression multinomiale ? </span> 

**On n'oubliera pas de ré-initialiser le modèle pour que l'apprentissage reprenne depuis le début.**

On ré-initialise le modèle.

In [ ]:
# re-create model to re-initialize parmaters 
mlp = Sequential()
mlp.add(Dense(64, activation='relu', input_dim=input_dim))
mlp.add(Dense(100, activation='relu'))
mlp.add(Dense(10, activation='softmax'))
mlp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 

On "re-fitte" le modèle avec 100 epochs et on représente l'évolution des performances d'apprentissage et de validation.

In [ ]:
# TODO : fit model with 100 epochs, save history and plot evolution of training / validation performance


On constate qu'en effet, le sur-apprentissage est plus fort qu'avec la régression multinomiale. On peut néanmoins constater que sur ce jeu de données cela ne semble pas être le cas.

In [ ]:
# TODO : compare performance obtainded with multinomial, mlp with 10 epochs and mlp with 100 epochs


## <span style="color:red"> $\Rightarrow$ POUR ALLER PLUS LOIN: jouer sur les paramètres du réseau (nombre de couches & nombre de neurones dans les couches) et mesurer l'impact sur les performances de classification.  </span> 

# Enfin, sauvegarder le modèle pour le réutiliser dans les notebooks à venir.

In [ ]:
mlp.save("model_mlp")